In [1]:
import numpy as np
from landlab import RasterModelGrid
from landlab.components import (FlowAccumulator, DepressionFinderAndRouter, Space, FastscapeEroder)
np.random.seed(seed = 5000)

In [2]:
mg = RasterModelGrid((5,5), xy_spacing=10.0)
_ = mg.add_zeros('topographic__elevation', at='node')
mg.at_node['topographic__elevation'] += (mg.node_y / 10. + mg.node_x / 10. + np.random.rand(len(mg.node_y)) / 10.)

mg.set_closed_boundaries_at_grid_edges(bottom_is_closed=True, left_is_closed=True,right_is_closed=True,top_is_closed=True)
mg.set_watershed_boundary_condition_outlet_id(0, mg.at_node['topographic__elevation'], -9999.)
fsc_dt = 100.
space_dt = 100.




In [3]:
fr = FlowAccumulator(mg, flow_director='D8')
df = DepressionFinderAndRouter(mg)
fsc = FastscapeEroder(mg, K_sp = .001,m_sp=.5, n_sp=1)


In [5]:
for x in range(100):
    fr.run_one_step()
    df.map_depressions()
    fsc.run_one_step(dt=fsc_dt)
    mg.at_node['topographic__elevation'][0] -= 0.001 # Uplift


In [7]:
_ = mg.add_zeros('soil__depth', at='node', dtype=float)
mg.at_node['soil__depth'] += 0.5
mg.at_node['topographic__elevation'] += mg.at_node['soil__depth']


FieldError: soil_depth@node

In [ ]:
ha = Space(mg, K_sed=0.00001,K_br=0.00000000001,F_f=0.5, phi=0.1, H_star=1., v_s=0.001, m_sp=0.5,  n_sp = 1.0, sp_crit_sed=0, sp_crit_br=0)


In [ ]:
for x in range(2000): #Space component loop
    fr.run_one_step()
    df.map_depressions()
    ha.run_one_step(dt=space_dt)
    mg.at_node['bedrock__elevation'][0] -= 2e-6 * space_dt